In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
import json
import pandas_bokeh
from bokeh.io import output_notebook, show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, Div,
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.models import Div, Paragraph, Row, Column
from bokeh.resources import CDN
from bokeh.util.browser import view
from jinja2 import Template

In [3]:
import os, glob
current_location = os.getcwd()
os.chdir(current_location)

In [4]:
# Read in shapefile
contiguous_usa = gpd.read_file("../shapefiles/cb_2018_us_state_20m.shp")

In [5]:
# Read csv files
df_covid = pd.read_csv("../data/raw_2_covid_latest.csv")
# Keep states which are in the shapefile contiguous_usa
df_covid = df_covid.loc[df_covid["State/Territory"].isin(contiguous_usa["NAME"])]

In [24]:
df_state = pd.read_csv("../data/raw_0_states.csv")
df_election_2016 = pd.read_csv("../data/raw_5_2016election.csv")

In [70]:
df_mail_2016 = pd.read_csv("../data/raw_6_1_2016mail.csv")
df_mail_2020 = pd.read_csv("../data/raw_8_2020_votebymail.csv")
df_mail_2020["Number absentee ballots returned"] = parse_str_int(df_mail_2020["Number absentee ballots returned"])

In [72]:
df_mail = pd.merge(left=df_mail_2016, right=df_mail_2020, left_on='State', right_on='State')

In [73]:
df_mail = pd.merge(left=df_mail, right=df_election_2020[["States", "total_2020"]], left_on='State', right_on='States')

In [74]:
df_mail = df_mail.drop("States", axis = 1)

In [76]:
df_mail.columns = ["state", "total_2016", "mail_2016", "percent_mail_2016", 
                   "mail_2020", "total_2020"]
df_mail["percent_mail_2020"] = (df_mail["mail_2020"]/df_mail["total_2020"])

In [77]:
df_mail

,state,total_2016,mail_2016,percent_mail_2016,mail_2020,total_2020,percent_mail_2020
0,Alabama,2137452.0,87553.0,0.0410,300000,2321377,0.129234
1,Alaska,323288.0,26750.0,0.0827,152000,358649,0.423813
2,Arizona,2722660.0,1991683.0,0.7315,2471000,3384904,0.730006
3,Arkansas,1048513.0,26655.0,0.0254,118000,1218149,0.096868
4,California,14610494.0,8453683.0,0.5786,12090000,17374657,0.695841
5,Colorado,2884199.0,2631744.0,0.9125,2809000,3256649,0.862543
6,Connecticut,1675955.0,129480.0,0.0773,636000,1825472,0.348403
7,Delaware,448217.0,13809.0,0.0308,148000,503857,0.293734
8,Florida,9613669.0,2657064.0,0.2764,4737000,11058549,0.428356
9,Georgia,4147161.0,199356.0,0.0481,1318000,4999004,0.263653


In [7]:
def parse_str_int(col):
    new_col = col.str.replace(',', '').astype(int)
    return new_col

df_election_2016["TRUMP_votes"] = parse_str_int(df_election_2016["TRUMP_votes"])
df_election_2016["CLINTON_votes"] = parse_str_int(df_election_2016["CLINTON_votes"])
df_election_2016["total_votes"] = parse_str_int(df_election_2016["total_votes"])

df_election_2016["2016_win"] = np.where(df_election_2016['TRUMP_votes'] > df_election_2016['CLINTON_votes'], 'Republican', "Democratic")
df_election_2016["2016_percent"] = (df_election_2016["CLINTON_votes"]-df_election_2016["TRUMP_votes"])/df_election_2016["total_votes"]
df_election_2016.head()

,state_code,TRUMP_election_votes,CLINTON_election_votes,TRUMP_votes,CLINTON_votes,Other_votes,total_votes,2016_win,2016_percent
0,AL,9,NaN,1318255,729547,"75,570",2123372,Republican,-0.277251
1,AK,3,NaN,163387,116454,"38,767",318608,Republican,-0.147306
2,AZ,11,NaN,1252401,1161167,"159,597",2573165,Republican,-0.035456
3,AR,6,NaN,684872,380494,"65,310",1130676,Republican,-0.269200
4,CA,NaN,55,4483814,8753792,"943,998",14181604,Democratic,0.301093


In [8]:
df_election_2020 = pd.read_csv("../data/raw_3_2020election.csv")
df_election_2020["2020_win"] = np.where(df_election_2020['BIDEN_percent'] > df_election_2020['TRUMP_percent'], 'Democratic', 'Republican')
df_election_2020["2020_percent"] = df_election_2020['BIDEN_percent'] - df_election_2020['TRUMP_percent']


In [42]:
df_election_2020["total_2020"] = df_election_2020["BIDEN_votes"]/df_election_2020["BIDEN_percent"]
df_election_2020["total_2020"] = df_election_2020["total_2020"].astype(int)

In [43]:
df_election_2020

,States,BIDEN_percent,BIDEN_votes,TRUMP_percent,TRUMP_votes,2020_win,2020_percent,color_2020,total_2020
0,Alabama,0.366,849624,0.620,1441170,Republican,-0.254,-6,2321377
1,Alaska,0.428,153502,0.528,189543,Republican,-0.100,-2,358649
2,Arizona,0.494,1672143,0.491,1661686,Democratic,0.003,2,3384904
3,Arkansas,0.348,423916,0.624,760613,Republican,-0.276,-8,1218149
4,California,0.636,11050282,0.343,5959245,Democratic,0.293,8,17374657
5,Colorado,0.554,1804184,0.419,1364467,Democratic,0.135,4,3256649
6,Connecticut,0.592,1080680,0.392,715291,Democratic,0.200,6,1825472
7,Delaware,0.588,296268,0.398,200603,Democratic,0.190,6,503857
8,District of Columbia,0.921,317323,0.054,18586,Democratic,0.867,8,344541
9,Florida,0.479,5297045,0.512,5668731,Republican,-0.033,-2,11058549


In [9]:
def color_category(col):
    
    r_25 = col[col<0].quantile(0.25)
    r_50 = col[col<0].quantile(0.50)
    r_75 = col[col<0].quantile(0.75)
    d_25 = col[col>=0].quantile(0.25)
    d_50 = col[col>=0].quantile(0.50)
    d_75 = col[col>=0].quantile(0.75)
    
    conditions = [
        (col<r_25),
        (col>=r_25) & (col<r_50),
        (col>=r_50) & (col<r_75),
        (col>=r_75) & (col<0),
        (col>=0) & (col<d_25),
        (col>=d_25) & (col<d_50),
        (col>=d_50) & (col<d_75),
        (col>=d_75)
    ]
    
    choices = [-8, -6, -4, -2, 2, 4, 6, 8]
    
    new_col = np.select(condlist=conditions, choicelist=choices)
    
    return new_col

In [10]:
df_election_2016["color_2016"] = color_category(df_election_2016["2016_percent"])
df_election_2020["color_2020"] = color_category(df_election_2020["2020_percent"])

In [11]:
df_election_2016 = pd.merge(left = df_election_2016[["state_code", "2016_win", "2016_percent", "color_2016"]], 
                            right = df_state[["state_code", "state"]],
                            left_on = 'state_code',
                            right_on = 'state_code')

In [12]:
df_merge = pd.merge(left=df_covid, right=df_election_2020, left_on='State/Territory', right_on='States')
df_merge = pd.merge(left=df_merge, right=df_election_2016, left_on='State/Territory', right_on='state')

In [13]:
# Merge shapefile with covid data
map_info = contiguous_usa.merge(df_merge, left_on = "NAME", right_on = "State/Territory")
# Drop Alaska and Hawaii
map_info = map_info.loc[~map_info['NAME'].isin(['Alaska', 'Hawaii'])]

In [14]:
df_merge.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,...,TRUMP_percent,TRUMP_votes,2020_win,2020_percent,color_2020,state_code,2016_win,2016_percent,color_2016,state
0,Alaska,27085,NaN,NaN,3845,3673,102,NaN,NaN,4,...,0.528,189543,Republican,-0.100,-2,AK,Republican,-0.147306,-4,Alaska
1,Alabama,234080,195887.0,38193.0,14848,4789,3459,3155.0,304.0,210,...,0.620,1441170,Republican,-0.254,-6,AL,Republican,-0.277251,-8,Alabama
2,Arkansas,146190,NaN,NaN,11842,4851,2387,NaN,NaN,162,...,0.624,760613,Republican,-0.276,-8,AR,Republican,-0.269200,-6,Arkansas
3,Arizona,302324,293574.0,8750.0,25412,4216,6464,5992.0,472.0,162,...,0.491,1661686,Democratic,0.003,2,AZ,Republican,-0.035456,-2,Arizona
4,California,1110370,NaN,NaN,81135,2807,18726,NaN,NaN,463,...,0.343,5959245,Democratic,0.293,8,CA,Democratic,0.301093,8,California


In [15]:
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = map_info.to_json())

In [194]:
#output_notebook()

Loading BokehJS ...

In [17]:
base_colors = ["#cb181d","#fb6a4a","#fcae91","#fee5d9","#eff3ff","#bdd7e7","#6baed6","#2171b5"]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = base_colors,
                                 low = df_merge["color_2020"].min(),
                                 high = df_merge["color_2020"].max())

# Define custom tick labels for color bar.
tick_labels = {'-8': 'Trump wins',
               '-6':'',
               '-4':'',
               '-2':'', 
               '2':'',
               '4':'',
               '6':'',
               '8':'Biden wins'}
# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 10,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)


# Create figure object
p = figure(title = 'COVID-19 cases & 2020 election', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'color_2020',
                                 'transform' : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),
                                  #('B_win_percent', '@B_win_percent'),
                                  #('Win', '@win'),
                                  ('Case Rate per 100000','@{Case Rate per 100000}'),
                                  ('Confirmed cases','@{Total Cases}'),
                                  ('Total deaths','@{Total Deaths}')]))
# Specify layout
p.add_layout(color_bar, 'below')

#show(p)

In [18]:
# scatter plot
#Create Div with DataFrame:
div_df = Div(text=df_merge.head(10).to_html(index=False), 
             width=550)

#Create Scatterplot:
p_scatter = df_merge.plot_bokeh(
    kind="scatter",
    x="Total Cases",
    y="Total Deaths",
    category="2020_win",
    title="test",
    show_figure=False)
'''
#Combine Div and Scatterplot via grid layout:
pandas_bokeh.plot_grid([[div_df, p_scatter]], 
                       plot_width=400, 
                       plot_height=350)
'''

'\n#Combine Div and Scatterplot via grid layout:\npandas_bokeh.plot_grid([[div_df, p_scatter]], \n                       plot_width=400, \n                       plot_height=350)\n'

In [19]:
template = Template(
    """
    <!DOCTYPE html>
    <html lang="en">
        <head>
            <meta charset="utf-8">
            <title>{{ title if title else "Bokeh Plot" }}</title>
            {{ bokeh_css | safe }}
            {{ bokeh_js | safe }}
        </head>
        <body>
            {{ plot_div | safe }}
            {{ plot_script | safe }}
        </body>
    </html> 
    """)

#p1 = figure(plot_width = 400, plot_height = 400)
#p2 = figure(plot_width = 400, plot_height = 400)
p3 = figure(plot_width = 800, plot_height = 400)
#p1.circle([1, 2, 3], [4, 5, 6])
#p2.line([1, 2, 3], [4, 5, 6])
p3.line([1, 2, 3], [4, 5, 6])

html = file_html(Column(Row(p), Row(p_scatter), Row(p3)), template = template, resources = CDN)

output_file = 'test_interaction.html'
with open(output_file, 'w') as f:
    f.write(html)
view(output_file)

In [21]:
swing_states = ["Arizona", "Colorado", "Florida", "Georgia", "Iowa", 
"Michigan", "Minnesota", "Nevada", "New Hampshire", 
"North Carolina", "Ohio", "Pennsylvania", "Texas", "Wisconsin"]